In [1]:
import keras
import pandas as pd
import os
import glob
import numpy as np
import sys
from keras import Input
import os
import json
import numpy as np
import math
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import Model
from keras.utils import np_utils
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
!pip install imblearn
!pip install pyampd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import statistics

Using TensorFlow backend.



You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


# new start

In [2]:
def label_gen_new(case_nos):
    h=[]
    label=[]
    hr=[]
    for case_no in case_nos:
        case=pd.read_csv('./processed_data/'+case_no+'.csv')
        np.set_printoptions(suppress=True)
        case=case.to_numpy()
        
        f = open('./label/'+case_no+'.txt', "r")
        case_label=f.read()
        case_label=case_label.split()
        case_label=np.array(case_label)
        case_label=case_label.astype(np.int)
        
        count=0
        #h=[]
        #label=[]
        for i in range(0,(case_label[case_label.shape[0]-1]),10):
            extract_pleth=case[i*100:i*100+1000,1]
            extract_hr=case[i*100:i*100+1000,2]
            if(0.0 not in extract_hr):
                if (i in case_label):
                    label.append(1)#bad signal=1
                else:
                    label.append(0)#good signal=1
                h.append(extract_pleth)
                hr.append(statistics.mean(extract_hr))
                count+=1
        print(case_no,count)
    h=np.asarray(h)
    label=np.asarray(label)#[np.newaxis,:].T
    hr=np.asarray(hr)[np.newaxis,:].T
    return h,label,hr

In [3]:
case=['Case01','Case02','Case15','Case16','Case17','Case18','Case19','Case20','Case21','Case22','Case23','Case24','Case25','Case32']

In [4]:
x_train,y_train,hr_train=label_gen_new(case)

Case01 721
Case02 101
Case15 82
Case16 1127
Case17 116
Case18 68
Case19 131
Case20 644
Case21 314
Case22 885
Case23 149
Case24 335
Case25 574
Case32 476


In [5]:
x_train

array([[0.3685 , 0.36801, 0.36728, ..., 0.53602, 0.53602, 0.53602],
       [0.53602, 0.53675, 0.53748, ..., 0.55018, 0.55678, 0.5641 ],
       [0.57094, 0.58266, 0.5873 , ..., 0.67643, 0.67595, 0.67546],
       ...,
       [0.36117, 0.35507, 0.35214, ..., 0.30159, 0.29695, 0.29231],
       [0.28767, 0.27863, 0.27448, ..., 0.29499, 0.29255, 0.29011],
       [0.28767, 0.28303, 0.28107, ..., 0.30452, 0.30134, 0.29817]])

In [6]:
hr_train

array([[55.261],
       [61.578],
       [59.966],
       ...,
       [65.781],
       [66.581],
       [66.238]])

In [7]:
x_train.shape

(5723, 1000)

In [8]:
y_train.shape

(5723,)

In [9]:
hr_train.shape

(5723, 1)

In [10]:
print(np.unique(y_train, return_counts=True))

(array([0, 1]), array([5349,  374]))


In [11]:
x_train=np.hstack((x_train,hr_train))

In [12]:
print("trainset before: ",np.unique(y_train, return_counts=True))
ros = RandomUnderSampler('auto')
x_train, y_train = ros.fit_sample(x_train, y_train)

print("trainset after: ",np.unique(y_train, return_counts=True))

trainset before:  (array([0, 1]), array([5349,  374]))
trainset after:  (array([0, 1]), array([374, 374]))


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=auto as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


In [13]:
hr_train=x_train[:,1000][np.newaxis,:]
x_train=x_train[:,0:1000]

In [14]:
hr_train.shape

(1, 748)

### Feature Engineering

In [15]:
!pip install peakutils

You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
import peakutils

In [17]:
def ampd(sigInput):
    """Find the peaks in the signal with the AMPD algorithm.

    Original implementation by Felix Scholkmann et al. in
    "An Efficient Algorithm for Automatic Peak Detection in
    Noisy Periodic and Quasi-Periodic Signals", Algorithms 2012,
    5, 588-603

    This Function was obtained from https://github.com/LucaCerina/ampdLib
    The original code and Licence are included in this repository

    Parameters
    ----------
    sigInput: ndarray
        The 1D signal given as input to the algorithm

    Returns
    -------
    pks: ndarray
        The ordered array of peaks found in sigInput
    """

    # Create preprocessing linear fit
    sigTime = np.arange(0, len(sigInput))

    fitPoly = np.polyfit(sigTime, sigInput, 1)
    sigFit = np.polyval(fitPoly, sigTime)

    # Detrend
    dtrSignal = sigInput - sigFit

    N = len(dtrSignal)
    L = int(np.ceil(N / 2.0)) - 1

    # Generate random matrix
    LSM = np.random.uniform(1.0, 2.0, size = (L,N)) # uniform + alpha = 1

    # Local minima extraction
    for k in np.arange(1, L):
        locMax = np.zeros(N, dtype=bool)
        mask = ((dtrSignal[k:N - k - 1] > dtrSignal[0: N - 2 * k - 1]) &
                (dtrSignal[k:N - k - 1] > dtrSignal[2 * k: N - 1]))
        mask = mask.flatten()

        locMax[k:N-k-1] = mask
        LSM[k - 1, locMax] = 0

    # Find minima
    G = np.sum(LSM, 1)
    l = np.where(G == G.min())[0][0]

    LSM = LSM[0:l, :]

    S = np.std(LSM, 0)

    pks = np.flatnonzero(S == 0)
    return pks

def get_ppg_peaks(ppg, use_ampd=True):
    if use_ampd:
        return ampd(ppg)
    filtered = np.zeros(np.array(ppg).shape)
    filtered[ppg > np.std(ppg)] = ppg[ppg > np.std(ppg)]

    ppg_peaks = peakutils.indexes(
        filtered, thres=0, min_dist=30
    )
    return ppg_peaks
def get_fft_peaks(fft, use_ampd=False):
    if use_ampd:
        return ampd(fft)
    #filtered = np.zeros(np.array(fft).shape)
    #filtered[fft > np.std(fft)] = fft[fft > np.std(fft)]

    fft_peaks = peakutils.indexes(
        fft, thres=0, min_dist=2
    )
    return fft_peaks
def no_of_peaks(ppg):
    indices=get_ppg_peaks(ppg,True)
    n_peak=len(indices)
    return n_peak

In [18]:
def get_max_val(ppg):
    x=max(ppg)
    return x
def get_min_val(ppg):
    x=min(ppg)
    return x

In [19]:
def vizualize_peaks(signal):
    indices=get_ppg_peaks(signal,True)
    i=np.empty(len(indices), dtype=float)
    for x in range(len(indices)):
        temp=indices[x]
        i[x]= signal[temp]

    fig, ax = plt.subplots()
    ax.plot(signal,color='r')
    ax.scatter(indices,i,color='b')
    plt.savefig('a.png')

In [20]:
def avg_peak_to_peak_distance(ppg):
    avg_peak_to_peak=0
    std_peak_to_peak=0
    sum_peak_to_peak=0
    indices=get_ppg_peaks(ppg,True)
    dif=[]
    
    if (len(indices)<2):
        avg_peak_to_peak=0
    else:
        for i in range(len(indices)-1):
            dif.append(indices[i+1]-indices[i])
            #print(i,"   ",indices[i+1]-indices[i])
        dif=np.asarray(dif)
        avg_peak_to_peak=sum(dif)/(len(indices)-1)
        std_peak_to_peak=np.std(dif)
        sum_peak_to_peak=sum(dif)
        
    return avg_peak_to_peak,std_peak_to_peak,sum_peak_to_peak

def std_signal(ppg):
    return(np.std(ppg))

def top_5_dominant_freq(signal):
    N=1000
    amplitude = np.fft.fft(signal)
    amplitude=np.abs(amplitude)[:N // 2]
    freq=np.fft.fftfreq(1000, d=0.01)[:N // 2]
    fft_peaks=get_fft_peaks(amplitude, use_ampd=False)


    peak=[]
    peak_freq=[]
    for i in range(len(fft_peaks)):
        peak.append(amplitude[fft_peaks[i]])
        frequencies = float("{:.2f}".format(freq[fft_peaks[i]]))
        peak_freq.append(frequencies)
    peak=np.asarray(peak)
    peak_freq=np.asarray(peak_freq)
    peak_sorted, peak_freq_sorted= zip(*sorted(zip(peak, peak_freq),reverse=True))
    top_five_freq=peak_freq_sorted[0:5]
#top_five_freq=sorted()
    top_five_freq=np.asarray(top_five_freq)
    
    return (peak_freq_sorted[0:5])

In [21]:
def top_5_dominant_freq(signal):
    N=1000
    amplitude = np.fft.fft(signal)
    amplitude=np.abs(amplitude)[:N // 2]
    freq=np.fft.fftfreq(1000, d=0.01)[:N // 2]
    fft_peaks=get_fft_peaks(amplitude, use_ampd=False)


    peak=[]
    peak_freq=[]
    for i in range(len(fft_peaks)):
        peak.append(amplitude[fft_peaks[i]])
        frequencies = float("{:.2f}".format(freq[fft_peaks[i]]))
        peak_freq.append(frequencies)
    peak=np.asarray(peak)
    peak_freq=np.asarray(peak_freq)
    peak=peak[0:5]
    peak_freq=peak_freq[0:5]
    
    return np.hstack((peak,peak_freq))

In [22]:
n_peak_train=[]
for i in (x_train):
    n_peak_train.append(no_of_peaks(i))

In [23]:
n_max_train=[]
for i in (x_train):
    n_max_train.append(get_max_val(i))

In [24]:
n_min_train=[]
for i in (x_train):
    n_min_train.append(get_min_val(i))

In [25]:
avg_peakdistance_train=[]
std_peakdistance_train=[]
sum_peakdistance_train=[]

for i,j in zip(n_peak_train,x_train):
    if (i==0):
        avg_peakdistance_train.append(0)
        std_peakdistance_train.append(0)
        sum_peakdistance_train.append(0)
        
    else:
        avg,std,summation=avg_peak_to_peak_distance(j)
        avg_peakdistance_train.append(avg)
        std_peakdistance_train.append(std)
        sum_peakdistance_train.append(summation)
        

In [26]:
std_train=[]
for i in (x_train):
    std_train.append(std_signal(i))

In [27]:
feat_freq=[]
for i in x_train:
    x=top_5_dominant_freq(i)
    feat_freq.append(x)

In [28]:
n_peak_train=np.asarray(n_peak_train)

n_max_train=np.asarray(n_max_train)

n_min_train=np.asarray(n_min_train)

avg_peakdistance_train=np.asarray(avg_peakdistance_train)
std_peakdistance_train=np.asarray(std_peakdistance_train)
sum_peakdistance_train=np.asarray(sum_peakdistance_train)

std_train=np.asarray(std_train)
feat_freq=np.asarray(feat_freq)

In [29]:
n_peak_train.shape

(748,)

In [30]:
hr_train.shape

(1, 748)

In [31]:
x_train_new=np.vstack((n_peak_train, hr_train)) #no_of_PPG_Peak,avg_hr

In [32]:
x_train_new=np.vstack((x_train_new,n_max_train)) #no_of_PPG_Peak,avg_hr,max

In [33]:
x_train_new=np.vstack((x_train_new,n_min_train))#no_of_PPG_Peak,avg_hr,max,min

In [34]:
x_train_new=np.vstack((x_train_new,avg_peakdistance_train))#no_of_PPG_Peak,avg_hr,max,min,Avg_p_p

In [35]:
x_train_new=np.vstack((x_train_new,std_peakdistance_train))#no_of_PPG_Peak,avg_hr,max,min,Avg_p_p,std_p_p

In [36]:
x_train_new=np.vstack((x_train_new,sum_peakdistance_train))#no_of_PPG_Peak,avg_hr,max,min,Avg_p_p,std_p_p,sum_p_p

In [37]:
x_train_new=np.vstack((x_train_new,std_train))

In [38]:
x_train_new.shape

(8, 748)

In [41]:
x_train.shape

(748, 1000)

In [40]:
feat_freq

array([[ 6.15809058, 62.80586043,  5.42169237, ...,  1.8       ,
         2.2       ,  2.5       ],
       [11.29832626, 12.11387081, 49.2245911 , ...,  0.9       ,
         1.5       ,  1.8       ],
       [21.38469089,  5.56363752,  7.69803693, ...,  1.1       ,
         1.5       ,  2.2       ],
       ...,
       [53.92115488, 39.8996201 , 60.92157514, ...,  1.1       ,
         1.5       ,  1.9       ],
       [26.73170758, 75.13557464,  5.16770169, ...,  1.5       ,
         2.3       ,  2.7       ],
       [49.79705311, 26.23955696, 81.61435036, ...,  1.2       ,
         1.9       ,  2.3       ]])

In [82]:
x_train_new=np.vstack((x_train_new,feat_freq.T))

In [83]:
x_train_new=x_train_new.T

In [84]:
y_train.shape

(748,)

In [39]:
final=pd.DataFrame({'No_of_PPG_peaks': x_train_new[:,0], 'AVG_HR': x_train_new[:,1], 'Max': x_train_new[:,2],'Min': x_train_new[:,3],'Avg_Peak_to_Peak_distance': x_train_new[:,4],'Std_Peak_to_Peak_distance': x_train_new[:,5],'Sum_Peak_to_Peak_distance': x_train_new[:,6],'Std_train_signal_itself': x_train_new[:,7],'amp1': x_train_new[:,8],'amp2': x_train_new[:,9],'amp3': x_train_new[:,10],'amp4': x_train_new[:,11],'amp5': x_train_new[:,12],'freq1': x_train_new[:,13],'freq2': x_train_new[:,14],'freq3': x_train_new[:,15],'freq4': x_train_new[:,16],'freq5': x_train_new[:,17],'output_label':y_train})








In [40]:
final

,No_of_PPG_peaks,AVG_HR,Max,Min,Avg_Peak_to_Peak_distance,Std_Peak_to_Peak_distance,Sum_Peak_to_Peak_distance,Std_train_signal_itself,amp1,amp2,amp3,amp4,amp5,freq1,freq2,freq3,freq4,freq5,output_label
0,13.0,87.850,0.73431,0.28474,70.750000,0.829156,849.0,0.100477,3.097244e+01,4.509540e+00,3.067070e+00,5.310319e+01,2.508092e+00,0.2,0.6,0.9,1.4,2.2,0
1,14.0,90.307,0.77680,0.24371,67.230769,0.421325,874.0,0.154121,9.454640e+00,9.908360e-01,2.661424e+00,6.857811e+00,9.796297e+01,0.3,0.6,0.9,1.2,1.5,0
2,17.0,104.000,0.76044,0.22271,57.812500,0.807678,925.0,0.166606,1.043699e+01,8.402099e+00,1.254805e+01,9.034826e+01,5.752346e+00,0.2,0.8,1.4,1.7,2.4,0
3,15.0,93.488,0.78974,0.25665,63.714286,0.958315,892.0,0.119793,3.438873e+01,5.640816e+00,1.338293e+01,5.810562e+01,4.848454e+00,0.2,0.7,1.3,1.6,2.2,0
4,8.0,55.000,0.75409,0.24444,109.714286,1.160577,768.0,0.161956,2.857532e+00,3.480889e+00,9.612737e+01,6.255263e+00,5.534817e+00,0.3,0.6,0.9,1.2,1.5,0
5,14.0,89.869,0.81050,0.25739,67.076923,0.997037,872.0,0.117179,3.097478e+01,4.763740e+00,6.685506e+01,1.648411e+00,1.567873e+00,0.2,0.8,1.5,2.1,2.4,0
6,9.0,56.000,0.77363,0.24322,108.375000,1.218349,867.0,0.120538,6.786681e+00,5.383240e+01,5.608160e+00,2.260894e+01,5.174697e+00,0.2,0.9,1.3,1.8,2.1,0
7,13.0,79.626,0.76435,0.23932,72.333333,2.211083,868.0,0.147264,6.558089e+00,5.071489e+00,8.601290e+01,4.993131e+00,3.751401e+01,0.4,0.7,1.4,1.9,2.8,0
8,8.0,55.846,0.77729,0.23980,110.142857,1.124858,771.0,0.139265,5.535101e+01,5.914623e+01,4.341619e+00,4.028002e+01,2.794219e+00,0.9,1.8,2.1,2.7,3.2,0
9,13.0,89.000,0.83394,0.28718,73.000000,19.621417,876.0,0.111514,2.333319e+01,1.656189e+01,1.232080e+01,5.576770e+01,6.746933e+00,0.2,0.8,1.1,1.5,2.0,0


In [41]:
final.to_csv('./finalprep/feature.csv',index=False)

In [42]:
signal=pd.DataFrame(x_train)








In [43]:
signal.to_csv('./finalprep/signal.csv',index=False)